# Borrower's Risk Analysis

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.
Your report will be considered when building a **credit score** for a potential customer. A **credit score** is used to evaluate the ability of a potential borrower to repay their loan.

## Description of the data

    children: the number of children in the family
    days_employed: how long the customer has been working
    dob_years: the customer’s age
    education: the customer’s education level
    education_id: identifier for the customer’s education
    family_status: the customer’s marital status
    family_status_id: identifier for the customer’s marital status
    gender: the customer’s gender
    income_type: the customer’s income type
    debt: whether the customer has ever defaulted on a loan
    total_income: monthly income
    purpose: reason for taking out a loan

## Step 1.Open the data file and have a look at general information

In [1]:
import pandas as pd
from io import BytesIO  
import requests
spreadsheet_id = '1pRv73thHcppT_Yszw8RRxNXKjVMPnOL6_9mQu9XLb8U'
file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
r = requests.get(file_name)
credit = pd.read_csv(BytesIO(r.content))
credit

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404100,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


## Step 2. Data Preprocessing

In [2]:
credit.head(10)        


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


First look, we can see that the data has negative values on days_employed column and there is a lot of duplicate values which basically saying the same thing like car purchase, purchase of a car which can just be combined to just one category.

In [3]:
credit.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949900,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423626,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400500,75.000000,4.000000,4.000000,1.000000,362496.645000


Taking a closer look at the data, children column have -1 value which can't be right and just possibly a typo and it also has a max value of 20 which could also be a typo and we can just replace that to 2. Days employed column has negative values which needs to be fixed as well.

In [4]:
#changing -1 to 1 from the children column
credit['children'] = credit['children'].abs()
#changing 20 to 2
credit.loc[credit['children']==20,'children']= 2

> Changing -1 to 1 from the children column and 20 to 2 because these are unusual and  clearly typos.

In [5]:
credit.query('days_employed > 0')['income_type'].value_counts()

retiree       3443
unemployed       2
Name: income_type, dtype: int64

In [6]:
#replacing positive days_employed with 0
credit.loc[credit['days_employed']> 0,'days_employed']= 0
credit.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,0.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


In [7]:
#changing the negative days_employed to positive 
credit['days_employed'] = credit['days_employed'].abs()


>  We can see that not only retirees has positive days_employed values but unemployed as well. We can pretty much differentiate that positive values indicates current unemployed status while negative values indicates currently employed.I replaced positive values on days_employed with 0 as it contains retirees and unemployed.I changed the negative values to positive indicating currently employed to differentiate the days_employed column.

In [8]:
#looking for missing values
print(credit.isnull().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


>  **2174** missing values for days_employed and total_income. This makes sense, if the individual has not been employed then they have no income.

In [9]:
#finding mean and median to determine what's best to fill in missing values
days_employed_mean = credit['days_employed'].mean()
days_employed_median = credit['days_employed'].median()
total_income_mean = credit['total_income'].mean()
total_income_median = credit['total_income'].median()
print('{:.2f}'.format(days_employed_mean))
print('{:.2f}'.format(days_employed_median))
print('{:.2f}'.format(total_income_mean))
print('{:.2f}'.format(total_income_median))

1934.12
1203.37
26787.57
23202.87


In [10]:
#filling in  missing values. 
credit['days_employed'] = credit['days_employed'].fillna(0)
credit['total_income'] = credit['total_income'].fillna(0)
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


> I decided to just fill in the missing values with 0. Missing values is only **10%** of the data and not going to skew the result much and will not affect the overall result. Assuming the missing values belongs to the data that defaulted, it will stil not affect the overall result but I decided to not drop them and just replace it with 0.  We have missing values because the user might have forgotten to fill it out or the way the data was imported.

In [11]:
#Replacing data type with integer type
credit['days_employed'] = credit['days_employed'].astype(int)
credit['total_income'] = credit['total_income'].astype(int)
credit.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [12]:
#finding duplicates
credit.duplicated().sum()


54

In [13]:
#dropping duplicates
credit = credit.drop_duplicates().reset_index(drop=True)

>   I identified **54** duplicates in the data or **less than 1% percent** which is not much. I decided to drop it.

In [14]:
print(credit['education'].unique())

["bachelor's degree" 'secondary education' 'Secondary Education'
 'SECONDARY EDUCATION' "BACHELOR'S DEGREE" 'some college'
 'primary education' "Bachelor's Degree" 'SOME COLLEGE' 'Some College'
 'PRIMARY EDUCATION' 'Primary Education' 'Graduate Degree'
 'GRADUATE DEGREE' 'graduate degree']


In [15]:
#Fixing duplicate values for education column
credit['education_lowercase'] = credit['education'].str.lower()
credit['education_lowercase'].value_counts()


secondary education    15188
bachelor's degree       5251
some college             744
primary education        282
graduate degree            6
Name: education_lowercase, dtype: int64

In [16]:
credit['education_lowercase'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

> Notice that education column is listed in uppercase and lowercase so I converted it to lower case to get the unique value. I also created a new column to store the lowercase values.


In [17]:
from nltk.stem import SnowballStemmer
english_stemmer = SnowballStemmer('english')
purpose_words= ['house', 'education', 'university','car', 'estate', 'property','wedding']
            
for word in purpose_words:
    print('source word - {},after stemming - {}'.format(word, english_stemmer.stem(word)))


source word - house,after stemming - hous
source word - education,after stemming - educ
source word - university,after stemming - univers
source word - car,after stemming - car
source word - estate,after stemming - estat
source word - property,after stemming - properti
source word - wedding,after stemming - wed


> Now if we look at the purpose column, there is multiple entries  that say 'car purchase','buying a second hand car' which we can condensed to just car purchase. There is also 'secondary education','university education' which we can just categorize as education. We need to create a new column for 'purpose' column to condense multiple entries that basically says the same thing.

In [18]:
from nltk.stem import SnowballStemmer
english_stemmer = SnowballStemmer('english')
queries = credit['purpose']

def purpose_newcategory(queries):
    for word in queries.split(' '):
        stemmed_word = english_stemmer.stem(word)
        if stemmed_word == 'hous':
            return 'housing'
        if stemmed_word == 'educ' or stemmed_word == 'univers':
            return 'education'
        if stemmed_word == 'car':
            return 'car purchase'
        if stemmed_word == 'estat' or stemmed_word == 'properti':
            return 'real estate'
        if stemmed_word == 'wed':
            return 'wedding'
    return 'other'
credit['purpose_new_category'] = credit['purpose'].apply(purpose_newcategory)
print(credit['purpose_new_category'].unique())

['housing' 'car purchase' 'education' 'wedding' 'real estate']


>  We need to create a function that will find the stem of the word from the 'purpose' column so we can combine the similar categories. And then we need to create a new column to store the results.


In [19]:
credit['purpose_new_category'].value_counts()

real estate     7005
car purchase    4308
education       4014
housing         3809
wedding         2335
Name: purpose_new_category, dtype: int64

>  And then we count how many entries represents each category. We use value_counts to get the count of unique values. Real estate make up 30% of loan purpose

In [20]:
credit['income_type'].value_counts()

employee                       11091
business                        5080
retiree                         3837
civil servant                   1457
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

>  Now we take a look at the income type.Employee makes up most of the data.

In [21]:
credit.groupby('debt')['children'].count()

debt
0    19730
1     1741
Name: children, dtype: int64

> There's **1741** who defaulted loan that has children compare to **19730** that don't. 1741 borrowers that defaulted is not much compared to 19730, 1741 is **8 percent of the data**.

## Step 3. Answer these questions:



####   Is there a connection between having kids and repaying a loan on time?

In [22]:
credit.groupby(['children','debt']).debt.sum()

children  debt
0         0          0
          1       1063
1         0          0
          1        445
2         0          0
          1        202
3         0          0
          1         27
4         0          0
          1          4
5         0          0
Name: debt, dtype: int64

In [23]:
credit['children'].describe()

count    21471.000000
mean         0.480229
std          0.755892
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          5.000000
Name: children, dtype: float64

In [24]:
def children_category(y):
    if y == 0:
        return 'Borrowers with no child'
    if y == 1:
        return 'Borrowers that have a child'
    else:
        return 'Borrowers that have more than 1 child'
credit['children_category'] = credit['children'].apply(children_category)


In [25]:
credit['children_category'].value_counts()

Borrowers with no child                  14107
Borrowers that have a child               4856
Borrowers that have more than 1 child     2508
Name: children_category, dtype: int64

In [26]:
credit.groupby(['children','debt']).debt.sum() / credit['debt'].sum()*100




children  debt
0         0        0.000000
          1       61.056864
1         0        0.000000
          1       25.560023
2         0        0.000000
          1       11.602527
3         0        0.000000
          1        1.550833
4         0        0.000000
          1        0.229753
5         0        0.000000
Name: debt, dtype: float64

> - These tell us how many borrowers have defaulted on a loan based on how many children are in a household. As we can see, **1063** defaulted on a loan with **0** children which is **61%** of the total borrowers that defaulted on a loan compared to the **39%** that did default and have kids. 
> - Most individuals with no child tend to borrow more than the ones that have children. **66%** borrowers have no children compare to the other **34%** that do.
> - These pretty much answers the question if there is a connection between having kids and repaying loans on time. And it also shows that the more children a borrower have, the less likely they are to default on a loan.

   #### Is there a connection between marital status and repaying a loan on time?

In [27]:
#categorizing family_status based on family_status_id
family_status_pivot = credit.pivot_table(index= 'family_status', values = 'family_status_id')
family_status_pivot

,family_status_id
family_status,
civil partnership,1
divorced,3
married,0
unmarried,4
widow / widower,2


In [28]:
credit.groupby(by= ['family_status','debt']).debt.sum()

family_status      debt
civil partnership  0         0
                   1       388
divorced           0         0
                   1        85
married            0         0
                   1       931
unmarried          0         0
                   1       274
widow / widower    0         0
                   1        63
Name: debt, dtype: int64

In [29]:
credit['family_status'].value_counts()

married              12344
civil partnership     4163
unmarried             2810
divorced              1195
widow / widower        959
Name: family_status, dtype: int64

As you can see, married people tend to default on a loan more often compared to unmarried people because married people tend to borrow more than the other familly status.

#### Is there a connection between income level and repaying a loan on time?

In [30]:
credit.groupby(by= ['income_type','debt']).debt.sum()

income_type                  debt
business                     0          0
                             1        376
civil servant                0          0
                             1         86
employee                     0          0
                             1       1061
entrepreneur                 0          0
paternity / maternity leave  1          1
retiree                      0          0
                             1        216
student                      0          0
unemployed                   0          0
                             1          1
Name: debt, dtype: int64

In [31]:
credit['total_income'].describe()

count     21471.000000
mean      24142.173909
std       17564.008086
min           0.000000
25%       14231.000000
50%       21714.000000
75%       31320.000000
max      362496.000000
Name: total_income, dtype: float64

In [32]:
def total_income_category(x):
    if x <20000:
        return 'Low Income'
    if 20000 < x < 40000:
        return 'Average Income'
    if 40000 < x < 75000:
        return "Above Average Income"
    else:
        return "High Income"
credit['total_income_category'] = credit['total_income'].apply(total_income_category)

In [33]:
display(credit['total_income_category'].value_counts())
credit.groupby(by= ['total_income_category', 'debt']).debt.sum()

Low Income              9489
Average Income          9169
Above Average Income    2524
High Income              289
Name: total_income_category, dtype: int64

total_income_category  debt
Above Average Income   0         0
                       1       178
Average Income         0         0
                       1       769
High Income            0         0
                       1        16
Low Income             0         0
                       1       778
Name: debt, dtype: int64

> - I categorized total_income to look on how income level affects > - Those who are employed and own a business tend to default on a loan, both combined making up 82.5% of the defaulted loans (376 + 1061 = 1437, 1437/1741 = 82.5%).
> - Low income and average income borrowers tend to borrow more which makes sense because they need the money but borrowers that have low income tend to default a bit more and average income coming in close to second. People does tend to spend more above their means.

In [34]:
credit.groupby(by= ['purpose_new_category','debt']).debt.sum()

purpose_new_category  debt
car purchase          0         0
                      1       403
education             0         0
                      1       370
housing               0         0
                      1       256
real estate           0         0
                      1       526
wedding               0         0
                      1       186
Name: debt, dtype: int64

> Defaulting on a loan based on its purpose is pretty close in all categories. Although from this data, we can conclude that borrowers of real etate and car purchase are less likely to repay their loans on time than the ones borrowing for  housing, wedding and education.

## Step 4. Overall conclusion

#### In conclusion:

> - Overall, there is a unsubtatantial correlation on defaulting on a loan based on children as it is only comprised of 8% of overall data. 
> - From this data, most of the borrowers do not have children. There could be a lot of different factors to keep in mind in combination with having children. Although the ones that did default, 61%  do not have children compare to 39% that do.
> - Married borrowers tend to default more than unmarried ones because they do tend to borrow more than the other group from that same category.
> - Low income earners tend to borrow more and default more
> - Borrowers of real estate and car purchase are less likely to repay their loans than the ones borrowing for other purposes
    
     
    